<a href="https://colab.research.google.com/github/LEECHANGDAEE/colab/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%84%EC%B2%98%EB%A6%ACnew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [116]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score, get_scorer
from sklearn.linear_model import Lasso, Ridge, LassoCV,LinearRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import KFold, RepeatedKFold, GridSearchCV, cross_validate, train_test_split

In [276]:

train = pd.read_csv('./train2.csv',encoding='cp949')
test = pd.read_csv('./test1.csv',encoding='cp949')
submission = pd.read_csv('./sample_submission1.csv',encoding='cp949')

train.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '신분',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수', '등록차량수'
]

test.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '신분',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수'
]

train= pd.get_dummies(data = train, columns = ['임대건물구분'], prefix = '건물구분')
test =  pd.get_dummies(data = test, columns = ['임대건물구분'], prefix = '건물구분')


In [277]:
data_m = train.groupby('단지코드').sum()
data_mt = test.groupby('단지코드').sum()

len(data_m)
data_st = data_m.건물구분_상가
data_ap = data_m.건물구분_아파트

data_stt = data_mt.건물구분_상가
data_apt = data_mt.건물구분_아파트

In [278]:
train.공급유형.value_counts()

국민임대         1727
임대상가          562
행복주택          202
공공임대(10년)     175
영구임대          152
공공임대(50년)      31
공공임대(분납)        7
공공분양            7
장기전세            3
공공임대(5년)        3
Name: 공급유형, dtype: int64

In [279]:
for i in range(len(train)) :
  if train.공급유형[i] in ['임대상가','영구임대'] :
    train.공급유형[i] = '영구임대_상가'
  elif train.공급유형[i] in ['공공임대(10년)','공공임대(50년)','공공임대(분납)','공공분양','공공임대(5년)'] :
    train.공급유형[i] = '공공임대'
  elif train.공급유형[i] in ['장기전세','행복주택'] :
    train.공급유형[i] = '전세주택'
train.공급유형.value_counts()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


국민임대       1727
영구임대_상가     714
공공임대        223
전세주택        205
Name: 공급유형, dtype: int64

In [280]:
for i in range(len(test)) :
  if test.공급유형[i] in ['임대상가','영구임대'] :
    test.공급유형[i] = '영구임대_상가'
  elif test.공급유형[i] in ['공공임대(10년)','공공임대(50년)','공공임대(분납)','공공분양','공공임대(5년)'] :
    test.공급유형[i] = '공공임대'
  elif test.공급유형[i] in ['장기전세','행복주택'] :
    test.공급유형[i] = '전세주택'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [281]:
test.공급유형.value_counts()

국민임대       622
영구임대_상가    222
전세주택       124
공공임대        54
Name: 공급유형, dtype: int64

In [282]:
train['전용면적'] = round(train['전용면적'])//5*5
test['전용면적'] = round(test['전용면적'])//5*5

idx = train[train['전용면적']>100].index
train.loc[idx, '전용면적'] = 1
idx = train[train['전용면적']>50].index
train.loc[idx, '전용면적'] = 2
idx = train[train['전용면적']==50].index
train.loc[idx, '전용면적'] = 3
idx = train[train['전용면적']>40].index
train.loc[idx, '전용면적'] = 4
idx = train[train['전용면적']>9].index
train.loc[idx, '전용면적'] = 5

idx = test[test['전용면적']>100].index
test.loc[idx, '전용면적'] = 1
idx = test[test['전용면적']>50].index
test.loc[idx, '전용면적'] = 2
idx = test[test['전용면적']==50].index
test.loc[idx, '전용면적'] = 3
idx = test[test['전용면적']>40].index
test.loc[idx, '전용면적'] = 4
idx = test[test['전용면적']>9].index
test.loc[idx, '전용면적'] = 5



columns = ['단지코드', '총세대수', '공가수', '지역', '단지내주차면수', '지하철', '버스','공급유형']
target = '등록차량수'
area_columns = []

for area in train['전용면적'].unique():
    area_columns.append(f'면적_{area}')

new_train = pd.DataFrame()
new_test = pd.DataFrame()

for i, code in tqdm(enumerate(train['단지코드'].unique())):
    temp = train[train['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_train.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_train.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()
    
    new_train.loc[i, '등록차량수'] = temp.loc[0, '등록차량수']
    
for i, code in tqdm(enumerate(test['단지코드'].unique())):
    temp = test[test['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_test.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_test.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()
  
new_train = new_train.fillna(0)
new_test = new_test.fillna(0)


411it [00:04, 92.86it/s]
150it [00:01, 92.25it/s]


In [283]:

nw = new_train.set_index('단지코드')
ntw = new_test.set_index('단지코드')


In [284]:
d1 = pd.concat([nw.iloc[:,:-1],data_ap],axis=1)
d2 = pd.concat([d1,data_st],axis=1)
d2 = d2.reset_index()

dt1 = pd.concat([ntw,data_apt],axis=1)
dt2 = pd.concat([dt1,data_stt],axis=1)
dt2 = dt2.reset_index()



In [285]:

x = d2.iloc[:,1:]
y = new_train['등록차량수']

In [286]:
yy = dt2.iloc[:,1:]

In [287]:
for i in range(len(x)) : 
  if x.지역[i] in ['서울특별시','부산광역시'] :
    x.지역[i] = '서부'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [288]:
for i in range(len(yy)) : 
  if yy.지역[i] in ['서울특별시','부산광역시'] :
    yy.지역[i] = '서부'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [289]:
# for i in range(len(x)) :
#      if x.지역[i] in ['광주광역시','부산광역시','대전광역시','울산광역시','대구광역시'] :
#     x.지역[i] = '광역시'
#   elif x.지역[i] in ['서울특별시','세종특별자치시','제주특별자치도'] :
#     x.지역[i] = '특별자치시'
#   elif x.지역[i] in ['경상남도','경상북도','충청북도','충청남도','전라남도','전라북도'] :
#     x.지역[i] = '지방시'

In [290]:
# for i in range(len(yy)) :
#   if yy.지역[i] in ['광주광역시','부산광역시','대전광역시','울산광역시','대구광역시'] :
#     yy.지역[i] = '광역시'
#   elif yy.지역[i] in ['서울특별시','세종특별자치시','제주특별자치도'] :
#     yy.지역[i] = '특별자치시'
#   elif yy.지역[i] in ['경상남도','경상북도','충청북도','충청남도','전라남도','전라북도'] :
#     yy.지역[i] = '지방시'

In [291]:
x = pd.get_dummies(data = x, columns = ['지역'], prefix = '지역')
x = pd.get_dummies(data = x, columns = ['공급유형'], prefix = '공급유형')
yy = pd.get_dummies(data = yy, columns = ['지역'], prefix = '지역')
yy = pd.get_dummies(data = yy, columns = ['공급유형'], prefix = '공급유형')

In [ ]:
for i in range(50) :
  
  x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, test_size=0.2,random_state =i)
  mlr = LinearRegression()
  mlr.fit(x_train, y_train)

  ridge = Ridge(alpha = 0.5).fit(x_train, y_train)

  lasso = Lasso(alpha=0.2).fit(x_train, y_train)

  y_linear_predict = mlr.predict(x_test)
  y_ridge_predict = ridge.predict(x_test)
  y_lasso_predict = lasso.predict(x_test)
  from sklearn.metrics import mean_absolute_error
  from sklearn.metrics import mean_squared_error
  print(mean_absolute_error(y_lasso_predict,y_test))

In [293]:

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, test_size=0.2,random_state =0)
mlr = LinearRegression()
mlr.fit(x_train, y_train)

ridge = Ridge(alpha = 0.5).fit(x_train, y_train)

lasso = Lasso(alpha=0.23).fit(x_train, y_train)

y_linear_predict = mlr.predict(x_test)
y_ridge_predict = ridge.predict(x_test)
y_lasso_predict = lasso.predict(x_test)
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
print('linear regression rmse : {}'.format(mean_squared_error(y_linear_predict,y_test)**0.5))
print('ridge regression rmse : {}'.format(mean_squared_error(y_ridge_predict,y_test)**0.5))
print('lasso regression rmse : {}'.format(mean_squared_error(y_lasso_predict,y_test)**0.5))
print(mean_absolute_error(lasso.predict(x_train),y_train))
print(mean_absolute_error(y_lasso_predict,y_test))


linear regression rmse : 125.6919713572535
ridge regression rmse : 125.43737962072765
lasso regression rmse : 125.27250419930648
111.28434498673153
90.25893018412157


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3229707.680120967, tolerance: 4977.526067987805
  positive)


In [294]:
yy.head()

,총세대수,공가수,단지내주차면수,지하철,버스,면적_5.0,면적_4.0,면적_3.0,면적_2.0,면적_1.0,건물구분_아파트,건물구분_상가,지역_강원도,지역_경기도,지역_경상남도,지역_경상북도,지역_광주광역시,지역_대구광역시,지역_대전광역시,지역_서부,지역_세종특별자치시,지역_울산광역시,지역_전라남도,지역_전라북도,지역_제주특별자치도,지역_충청남도,지역_충청북도,공급유형_공공임대,공급유형_국민임대,공급유형_영구임대_상가,공급유형_전세주택
0,754.0,14.0,683.0,0.0,2.0,116.0,262.0,376.0,0.0,0.0,8,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,1354.0,9.0,1216.0,0.0,3.0,420.0,356.0,578.0,0.0,0.0,9,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,619.0,18.0,547.0,0.0,16.0,247.0,196.0,132.0,44.0,0.0,9,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
3,593.0,7.0,543.0,0.0,3.0,335.0,174.0,84.0,0.0,0.0,4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
4,1297.0,11.0,1112.0,0.0,2.0,425.0,568.0,304.0,0.0,0.0,8,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [258]:
type(yy.지역_서울특별시[0])

numpy.int64

In [295]:
y_lasso_predict = lasso.predict(yy)
submission['num'] = y_lasso_predict

submission.to_csv('lasso.csv', index=False)